In [ ]:
def load_mask(self, image_id):

        info = self.image_info[image_id]
        json_path = info["path"].parent / (info["path"].stem + ".json")
        with open(json_path) as f:
            d = json.load(f)
            data = pd.DataFrame.from_dict(d["microtubule"])

        def get_line(x):
            d = {}
            d["start_x"] = x[x.type == "seed"]["start_x"].values[0]
            d["start_y"] = x[x.type == "seed"]["start_y"].values[0]
            d["end_x"] = x[x.type == "seed"]["end_x"].values[0]
            d["end_y"] = x[x.type == "seed"]["end_y"].values[0]
            return pd.DataFrame([d])

        def draw_line(image, line, line_thickness):
            line = np.round(line).astype("int16")
            p1 = (line["start_x"], line["start_y"])
            p2 = (line["end_x"], line["end_y"])
            _, p1, p2 = cv2.clipLine((0, 0, image.shape[0], image.shape[1]), p1, p2)
            image = cv2.line(image, p1, p2, (1,), line_thickness)
            return image

        lines = data.groupby("mt_id").apply(get_line).reset_index(drop=True)
        count = lines.shape[0]

        mask = np.zeros([info['width'], info['height'], count], dtype=np.uint8)
        for i, line in lines.iterrows():
            mask[:, :, i] = draw_line(mask[:, :, i].copy(), line, info["line_thickness"])

        # Handle occlusions
        handle_occlusion = True
        if handle_occlusion:
            occlusion = np.logical_not(mask[:, :, -1]).astype(np.uint8)
            for i in range(count - 2, -1, -1):
                mask[:, :, i] = mask[:, :, i] * occlusion
                occlusion = np.logical_and(occlusion, np.logical_not(mask[:, :, i]))

        # Map class names to class IDs (all "microtubule" here).
        class_ids = np.repeat(self.class_names.index("microtubule"), count)

        return mask.astype(np.bool), class_ids.astype(np.int32)